In [ ]:
#!pip install git+https://github.com/HolyBayes/pytorch_ard

## Dataset

In [ ]:
import torch_ard as nn_ard
from torch import nn
import torch
import torch.nn.functional as F
from torch.nn import Parameter

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, LabelBinarizer
from sklearn.metrics import roc_auc_score

from torch_ard import get_ard_reg, get_dropped_params_ratio, ELBOLoss
from tqdm import trange, tqdm

import numpy as np

import time

In [ ]:
data = pd.read_csv("PID-dataset-subsample.csv")

le = LabelEncoder()
oh = OneHotEncoder()

X = data.drop(columns = ["Label"])

y_init = data["Label"]

le.fit(y_init)
y = le.transform(y_init)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.65, random_state=42, stratify=y)

y_testL = le.inverse_transform(y_test)

ss = StandardScaler()
ss.fit(X_train)
X_train = ss.transform(X_train)
X_test = ss.transform(X_test)


In [ ]:
oh = OneHotEncoder()
oh.fit(y.reshape(-1, 1));

In [ ]:
y_train = oh.transform(y_train.reshape(-1, 1)).toarray()
y_test =  oh.transform(y_test.reshape(-1, 1)).toarray()

In [ ]:
y_train = torch.from_numpy(np.array(y_train)).float()
y_test = torch.from_numpy(np.array(y_test)).float()

## Training

In [ ]:
torch.manual_seed(42)

In [ ]:
device = "cpu"

In [ ]:
c


In [ ]:
class DenseModelARD(nn.Module):
    def __init__(self, input_shape, output_shape, hidden_size=150, activation=None):
        super(DenseModelARD, self).__init__()
        self.l1 = nn_ard.LinearARD(input_shape, hidden_size)
        self.l2 = nn_ard.LinearARD(hidden_size, output_shape)
        self.activation = activation
        self._init_weights()

    def forward(self, input):
        x = input.to(self.device)
        x = self.l1(x)
        x = nn.functional.tanh(x)
        x = self.l2(x)
        if self.activation: x = self.activation(x)
        return x

    def _init_weights(self):
        for layer in self.children():
            if hasattr(layer, 'weight'): nn.init.xavier_uniform(layer.weight, gain=nn.init.calculate_gain('relu'))

    @property
    def device(self):
        return next(self.parameters()).device

In [ ]:
X_train.shape

(39000, 49)

In [ ]:
model = DenseModelARD(input_shape=X_train.shape[1], output_shape=6,
                      activation=nn.functional.relu).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, 'min')
criterion = ELBOLoss(model, F.mse_loss).to(device)

<ipython-input-10-83767fc1a9ba>:19: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  if hasattr(layer, 'weight'): nn.init.xavier_uniform(layer.weight, gain=nn.init.calculate_gain('relu'))


In [ ]:
X_train, X_test, y_train, y_test = \
    [torch.from_numpy(np.array(x)).float().to(device)
     for x in [X_train, X_test, y_train, y_test]]
X_train, X_test, y_train, y_test = torch.Tensor(X_train), torch.Tensor(X_test), torch.Tensor(y_train), torch.Tensor(y_test)
n_epoches = 1000
debug_frequency = 100
def get_kl_weight(epoch): return min(1, 2 * epoch / n_epoches)


pbar = trange(n_epoches, leave=True, position=0)
for epoch in pbar:
    kl_weight = get_kl_weight(epoch)
    opt.zero_grad()
    preds = model(X_train).squeeze()
    loss = criterion(preds, y_train, 1, kl_weight)
    loss.backward()
    opt.step()
    loss_train = float(
        criterion(preds, y_train, 1, 0).detach().cpu().numpy())
    preds = model(X_test).squeeze()
    loss_test = float(
        criterion(preds, y_test, 1, 0).detach().cpu().numpy())
    pbar.set_description('MSE (train): %.3f\tMSE (test): %.3f\tReg: %.3f\tDropout rate: %f%%' % (
        loss_train, loss_test, get_ard_reg(model).item(), 100 * get_dropped_params_ratio(model)))
    pbar.update()

MSE (train): 0.153	MSE (test): 0.153	Reg: 0.199	Dropout rate: 100.000000%: 100%|██████████| 1000/1000 [05:20<00:00,  3.12it/s]


In [ ]:
start_time = time.time()
pred = model(X_test)
t = time.time() - start_time
print(t)

0.06588244438171387


In [ ]:
pred = pd.DataFrame(pred.detach().numpy())
pred = pred.idxmax(axis = 1)
pred = le.inverse_transform(pred)
lb = LabelBinarizer().fit(y_testL)

In [ ]:
for label in ['Electron', 'Ghost', 'Kaon', 'Muon', 'Pion', 'Proton']:
    class_id = np.flatnonzero(lb.classes_ == label)[0]
    print(f"{label} score is {roc_auc_score(lb.transform(y_testL)[:, class_id], lb.transform(pred)[:, class_id])}")

Electron score is 0.4967564345710528
Ghost score is 0.503674278511097
Kaon score is 0.5
Muon score is 0.5002155767468908
Pion score is 0.5
Proton score is 0.5
